In [1]:
from utils import *
from sknn_lenskit import *
import pandas as pd
import numpy as np
import time
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm
from tqdm import tqdm

In [2]:
FOLDER_TRAIN = 'data/train/'
FOLDER_TEST = 'data/test/'
train, test = load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 0.562955379486084s
 -- filtered in: 0.8627009391784668s
 -- actions:  3344374
 -- items:  461880
 -- lists:  50000


In [3]:
df = train['actions'][['playlist_id', 'track_id']]
df['track_id'] -= 1
df['rating'] = np.ones(df.shape[0])
df = df.rename(columns={"playlist_id": "user", "track_id": "item"})

C:\Users\nanic\AppData\Local\Temp\ipykernel_5000\493546403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['track_id'] -= 1
C:\Users\nanic\AppData\Local\Temp\ipykernel_5000\493546403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = np.ones(df.shape[0])


In [4]:
rec = UserUser(2000, feedback='implicit')

In [5]:
start = time.time()
rec = rec.fit(df)
end = time.time()
print('Training Time: {}'.format(end - start))

Mapping build time:  0.7815060615539551
IDF calculate time:  0.3339078426361084
Training Time: 2.2486817836761475


In [7]:
n = []
s = []
p = []
pids = np.random.choice(train['playlists']['playlist_id'].values, 100)
items = np.unique(df['item'].values)
for pid in tqdm(pids):
    start = time.time()
    _, n_time, s_time = rec.predict_for_user(pid, items)
    end = time.time()
    p.append(end - start)
    n.append(n_time)
    s.append(s_time)
print('Mean find neighbor time: ', np.mean(n))
print('Mean score item time: ', np.mean(s))
print('Mean prediction time: ', np.mean(p))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.13it/s]

Mean find neighbor time:  0.025426249504089355
Mean score item time:  0.04332418918609619
Mean prediction time:  0.07554686307907105
